In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Tuple

In [2]:
# load data
df = pd.read_csv("train.csv")
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# data preprocessing
df = df[["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"]]
df["Age"].fillna(df["Age"].median(), inplace=True)
df["Sex"] = df["Sex"].map({"male": 0, "female": 1})

C:\Users\TUAN NGOC\AppData\Local\Temp\ipykernel_12072\3398935438.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Age"].fillna(df["Age"].median(), inplace=True)


In [4]:
# manual-normalize min-max
def normalize(df):
    return (df - df.min()) / (df.max() - df.min())

X = df.drop("Survived", axis=1).values
# numpy array
y = df["Survived"].values.reshape(-1, 1)
X = normalize(pd.DataFrame(X)).values
print(X)

[[1.         0.         0.27117366 0.125      0.         0.01415106]
 [0.         1.         0.4722292  0.125      0.         0.13913574]
 [1.         1.         0.32143755 0.         0.         0.01546857]
 ...
 [1.         1.         0.34656949 0.125      0.33333333 0.04577135]
 [0.         0.         0.32143755 0.         0.         0.0585561 ]
 [1.         0.         0.39683338 0.         0.         0.01512699]]


In [5]:
# train_test_split
def train_test_split(X, y, test_size=0.2, random_state=42):
    np.random.seed(random_state)
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)

    test_count = int(X.shape[0] * test_size)
    train_index = indices[:-test_count]
    test_index = indices[-test_count:]

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
class LogictisRegression:
    def __init__(self, learning_rate=0.0001, iterations=1000, convergence_tol=1e-6):
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.convergence_tol = convergence_tol
        self.w = None
        self.b = None
        self.losses = []

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def compute_loss(self, y_true, y_pred):
        e = 1e-8
        return -np.mean(y_true * np.log(y_pred + e) + (1 - y_true) * np.log(1 - y_pred + e))

    def fit(self, X, y):
        self.w = np.zeros((X.shape[1], 1))
        self.b = 0

        for iters in range(self.iterations):
            y_pred = self.predict(X)
            error = y_pred - y
            loss = self.compute_loss(y, y_pred)
            self.losses.append(loss)

            if iters > 0 and abs(self.losses[-1] - self.losses[-2]) < self.convergence_tol:
                break

            dw = X.T @ error / X.shape[0]
            db = np.mean(error)

            self.w -= self.learning_rate * dw
            self.b -= self.learning_rate * db

    def predict(self, X):
        z = X @ self.w + self.b
        return self.sigmoid(z)
    
    def evaluate(self, X, y):
        y_pred = self.predict(X)
        y_true = (y_pred >= 0.5).astype(int)
        accuracy = (y_true == y).mean()
        return accuracy

In [9]:
logictis_regression = LogictisRegression(learning_rate=0.0001, iterations=1000, convergence_tol=1e-6)
logictis_regression.fit(X_train, y_train)

acc_train = logictis_regression.evaluate(X_train, y_train)
acc_test = logictis_regression.evaluate(X_test, y_test)

print(f"Train Acc: {acc_train:.4f}")
print(f"Test Acc: {acc_test:.4f}")

Train Acc: 0.3885
Test Acc: 0.3652
